#### Week 1: Building Advanced RAG Applications.  Authored by Chris Sanchez.

# Week 1 - Notebook 3

# Overview
***
Welcome to the final notebook for Week 1! Take a look at all the ground we've covered so far:
- Chunking/splitting
- Text vectorization
- Combining metadata
- Collection Configuration
- Data Indexing
- Keyword search
- Vector search
- OPTIONAL: Searching with Filters

We are now prepared to move on to a very important topic, **Retrieval Evaluation**.  I hope you've noticed that the search results will differ (sometimes slightly, sometimes by a lot) depeding on which search method you used: `keyword_search` or `vector_search`.  As humans, it's fairly easy for us to determine whether the returned search results are relevant to the query that was submitted, (though even here there will be differing opinions on result relevance).  But how do we systematically determine which search method is better in general?  And how do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?

We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that.  I say "one way" because there are many ways to approach this problem, and the method I'm showing you is not perfect (if anything it's a bit too conservative).  Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, and absent any user [click-data](https://en.wikipedia.org/wiki/Click_tracking), requires some form of data labeling.  With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier. Let's take a look at how that works.

# Retrieval Evaluation - Process
***
Here's a high-level overview of how the Retrieval Evaluation process in this notebook works:

1. Generate a "golden dataset" of query-context pairs.  I've already pre-generated multiple golden datasets for our course. I randomly selected 100 document chunks (contexts) from the Huberman Lab corpus and those chunks were then submitted to the `gpt-3.5-turbo` model which generated queries that can be answered by the context.  The output was 100 query-context pairs along with their associated doc_ids. 
   - **Baseline Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateWCS`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from the Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant doc_id is found in the list of retrieved results doc_ids, for each query.

#### Problems with this Approach
The problems with this approach are many, I'll cover a few here:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are generated by `gpt-3.5-turbo`, I think the first assumption will generally be true.  When reviewing the dataset I did find a few questions that were not answerable given the context, but for the most part they were.  The 2nd assumption though, is going to be dependent on your particular search use case.  I think for the purposes of this course, the questions generated are a decent reflection of how someone would query this dataset, and therefore do the job of measuring retriever performance.  But I would always check a real-world query distribution before using an approach like the one presented here.
- This approach is conversative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we have no way to account for these other relevant documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the ["hit rate"](https://uplimit.com/course/rag-applications/v2/module/retreival-evaluation#corise_clp66zqui003i2a777aldseor) which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and [Mean Reciprocal Rank (MRR)](https://uplimit.com/course/rag-applications/v2/module/retreival-evaluation#corise_clp66zqui003j2a77u8lnrk5b).  We're using MRR instead of other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

In [1]:
#standard library imports
import sys
sys.path.append('../')

from typing import Any
import time
import os

# utilities
from tqdm import tqdm
from rich import print
from dotenv import load_dotenv, find_dotenv
env = load_dotenv(find_dotenv(), override=True)

# Assignment 1.3
***
#### Instructions:
* Import the `/data/golden_datasets/golden_256.json` dataset using the `load_json` method of the FileIO Class
* Instantiate a new WeaviateWCS client (Retriever) and set the `collection_name` of the Collection that you created in Notebook 2
* Evaluate your retriever results using the `retrieval_evaluation` function
* Submit your results in the form of a text file to Uplimit (the function autogenerates a report in the `dir_outpath` directory).

In [2]:
from src.evaluation.retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.database.database_utils import get_weaviate_client
from src.preprocessor.preprocessing import FileIO


data_path = '../data/golden_datasets/golden_256.json'

#################
##  START CODE ##
#################


### Load QA dataset
golden_dataset = FileIO.load_json("../data/golden_datasets/golden_256.json")

### Instantiate Weaviate client and set Collection name
#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']
model_path = 'sentence-transformers/all-MiniLM-L6-v2'

retriever = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)
collection_name = "Huberman_minilm_256"


#################
##  END CODE   ##
#################

# should see 100 queries
print(f'Num queries in Golden Dataset: {len(golden_dataset["queries"])}')

/Users/neerajprad/miniconda3/envs/vs-uplimit/lib/python3.10/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/neerajprad/miniconda3/envs/vs-uplimit/lib/python3.10/site-packages/litellm/proxy/_types.py:167: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @root_validator(pre=True)
/Users/neerajprad/miniconda3/envs/vs-uplimit/lib/python3.10/site-packages/litellm/proxy/_types.py:254: PydanticDeprecatedSince20: `pydantic.con

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fd6726511e0>


Num queries in Golden Dataset: 100

In [3]:
golden_dataset.keys()

dict_keys(['queries', 'corpus', 'relevant_docs'])

In [4]:
golden_dataset['queries']

{'92ea1b3e-6c98-40d5-ba54-5bf7f98a5a20': 'How can one deliberately increase adrenaline while staying calm mentally, and what utility does this have in dealing with unwanted events?',
 'b1cb4d2a-9a81-4701-a481-ba3ab870106d': 'What behavioral approach can be used to deliberately turn off the communication between sensory neurons and motor neurons for pain relief?',
 'a217d609-7e74-499f-831f-c6c4345bb44a': "How has Tim Ferriss's approach changed since writing The 4-Hour Body, in terms of optimizing and de-optimizing certain areas for well-being?",
 'bca8f347-5437-44f6-818e-ab55b32b7640': 'How can visualizing failure be an effective strategy to motivate oneself when feeling unmotivated towards a goal?',
 '06d9e817-17bd-4e20-828e-aa034f816f3b': 'How do people find a comfortable level when taking betaine HCL pepsin tablets or capsules?',
 'e3d05215-3c7d-4eb8-ba38-519bc296f5d6': 'How does the level of cortisol released at baseline relate to the effects of alcohol consumption on stress?',
 'db

In [5]:
len(golden_dataset['relevant_docs']), len(golden_dataset['corpus'])

(100, 77)

# Project 1: Retrieval Evaluation

In [6]:
def retrieval_evaluation(dataset: dict,
                         collection_name: str,
                         retriever: WeaviateWCS,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         query_properties: list[str]=['content'],
                         return_properties: list[str]=['doc_id', 'content'],
                         dir_outpath: str='./eval_results',
                         include_miss_info: bool=False
                         ) -> dict[str, str|int|float]:
    '''
    Given a dataset and a retriever evaluate the performance of the retriever. Returns a dict of kw and vector
    hit rates and mrr scores. If inlude_miss_info is True, will also return a list of kw and vector responses
    and their associated queries that did not return a hit, for deeper analysis. Text file with results output
    is automatically saved in the dir_outpath directory.

    Args:
    -----
    dataset: dict
        Dataset to be used for evaluation
    collection_name: str
        Name of Collection on Weaviate host to be used for retrieval
    retriever: WeaviateWCS
        WeaviateWCS object to be used for retrieval
    retrieve_limit: int=5
        Number of documents to retrieve from Weaviate host, increasing this value too high
        will artificially inflate the hit rate score of your retriever.
    chunk_size: int=256
        Number of tokens used to chunk text. This value is purely for results
        recording purposes and does not affect results.
    return_properties: list[str]=['doc_id', 'content']
        list of properties to be returned from Weaviate host for display in response
    dir_outpath: str='./eval_results'
        Directory path for saving results.  Directory will be created if it does not
        already exist.
    include_miss_info: bool=False
        Option to include queries and their associated kw and vector response values
        for queries that are "total misses"
    '''

    results_dict = {'n':retrieve_limit,
                    'Retriever': retriever.model_name_or_path,
                    'chunk_size': chunk_size,
                    'query_props': query_properties,
                    'kw_hit_rate': 0,
                    'kw_mrr': 0,
                    'vector_hit_rate': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }

    start = time.perf_counter()
    miss_info = []
    for query_id, q in tqdm(dataset['queries'].items(), 'Queries'):
        results_dict['total_questions'] += 1
        hit = False

        try:
            kw_response = retriever.keyword_search(request=q, collection_name=collection_name, query_properties=query_properties,
                                                   limit=retrieve_limit, return_properties=return_properties)
            vector_response = retriever.vector_search(request=q, collection_name=collection_name,
                                                   limit=retrieve_limit, return_properties=return_properties)

            #collect doc_ids and position of doc_ids to check for document matches
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}

            #extract doc_id for scoring purposes
            doc_id = dataset['relevant_docs'][query_id]

            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_hit_rate'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
                hit = True
            if doc_id in vector_doc_ids:
                results_dict['vector_hit_rate'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
                hit = True

            # if no hits, let's capture that
            if not hit:
                results_dict['total_misses'] += 1
                miss_info.append({'query': q, 'kw_response': kw_response, 'vector_response': vector_response})
        except Exception as e:
            print(e)
            continue


    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict, search_type=['kw', 'vector'])
    calc_mrr_scores(results_dict, search_type=['kw', 'vector'])

    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, dir_outpath=dir_outpath, as_text=True)

    if include_miss_info:
        return results_dict, miss_info
    return results_dict

### Run evaluation over golden dataset

In [7]:
#################
##  START CODE ##
#################
eval_results = retrieval_evaluation(dataset=golden_dataset, collection_name=collection_name, retriever=retriever, include_miss_info=True)

Queries: 100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


Total Processing Time: 0.74 minutes

In [9]:
print(eval_results)

(
    {
        'n': 5,
        'Retriever': 'sentence-transformers/all-MiniLM-L6-v2',
        'chunk_size': 256,
        'query_props': ['content'],
        'kw_hit_rate': 0.9,
        'kw_mrr': 0.82,
        'vector_hit_rate': 0.71,
        'vector_mrr': 0.59,
        'total_misses': 8,
        'total_questions': 100
    },
    [
        {
            'query': 'How can listeners support the podcast in addition to leaving reviews and comments on Spotify 
and Apple?',
            'kw_response': [
                {
                    'content': "Nonetheless, I think many people who are listeners of this podcast are interested 
in the things that they can do in order to inoculate themselves or at least reduce the likelihood of experiencing 
headache, especially for people who are experiencing chronic recurring headaches, such as migraine or the other 
forms of headache, which can be so debilitating. Thank you for joining me for today's discussion. If you're 
learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's the best zero cost way 
to support us. In addition, please subscribe to the podcast on Spotify and Apple. And on both Spotify and Apple, 
you can leave us up to a five-star review. If you have questions for me or comments about the podcast or guests 
that you'd like me to include on the Huberman Lab podcast, please put all of that in the comment section on 
YouTube. I do read all the comments. In addition, please check out the sponsors mentioned at the beginning and 
throughout today's episode. That's the best way to support this podcast. If you're not already following us on 
social media, we are Huberman Lab on Instagram, Twitter, Facebook, and LinkedIn.",
                    'doc_id': 'CGjdgy0cwGk_144',
                    'score': 17.35101890563965
                },
                {
                    'content': "And be sure to check out the Waking Up app that Dr. Sam Harris has made free to any
Huberman Lab listeners for 30 days by going to wakingup.com slash Huberman. Please also check out his incredible 
podcast, the Making Sense podcast. And you can find any number of Sam Harris's different books on meditation, 
consciousness, philosophy, neuroscience, politics, and more. You can find links to those books by going to 
samharris.org. If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. 
That's the best zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And
on both Spotify and Apple, you can leave us up to a five-star review. If you have questions for us or comments or 
topics that you'd like me to cover, or guests you'd like me to invite onto the Huberman Lab podcast, please put 
those in the comment section on YouTube. I do read all the comments. Please also check out the sponsors mentioned 
at the beginning and throughout today's episode. That's the best way to support this podcast.",
                    'doc_id': '-wIt_WsJGfw_228',
                    'score': 16.914188385009766
                },
                {
                    'content': "Susanna Soberg. If you'd like to learn more about Dr. Soberg's research, or you 
would like to learn about the research of her institute, the Soberg Institute, please see the links in the show 
note caption. Also in the show note caption, you can find a link to Dr. Soberg's excellent book, Winter Swimming. 
If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's a terrific 
zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And in addition, on
both Spotify and Apple, you can leave us up to a five-star review. If you have questions for me or topics you'd 
like me to cover on the Huberman Lab podcast, or guests that you'd like me to consider inviting on the Huberman Lab
podcast, please put that in the comments on YouTube. I do read all the comments. In add

In [31]:
for q in eval_results[1]:
    print(q["query"])
    query_id = [k for k, v in golden_dataset['queries'].items() if v == q["query"]][0]
    related_doc = golden_dataset['relevant_docs'][query_id]
    print(golden_dataset['corpus'][related_doc])

How can listeners support the podcast in addition to leaving reviews and comments on Spotify and Apple?

And now on both Spotify and Apple, you can leave us up to a five-star review. Please also leave us comments and 
feedback in the comment section on our YouTube channel. You can also suggest future guests that you'd like us to 
cover. We do read all those comments. Please also check out the sponsors mentioned at the beginning of today's 
podcast. That's a terrific, perhaps the best way to support this podcast. We also have a Patreon, it's patreon.com 
slash Andrew Huberman, and there you can support this podcast at any level that you like. During today's episode 
and on many previous episodes of the Huberman Lab Podcast, we discuss supplements. While supplements aren't 
necessary for everybody, many people derive tremendous benefit from them for things like enhancing sleep and focus,
and indeed for learning and memory. For that reason, the Huberman Lab Podcast is now partnered with Momentous 
Supplements. The reason we partnered with Momentous is several fold. First of all, we wanted to have one location 
where people could go to access single ingredient, high quality versions of the supplements that we were discussing
on this podcast. This is a critical issue.

Why is it recommended to delay caffeine intake for 90 to 120 minutes after waking up, and how does this relate to 
clearing adenosine in the morning?

And so because caffeine acts as an adenosine antagonist effectively, while caffeine is present in those receptors, 
you don't feel as sleepy. You have more energy. Your reaction time goes down. Memory is enhanced, focus, 
performance of all kinds. Yes, but then when that caffeine is dislodged from the receptor, then the adenosine can 
act even more potently at those receptors. So it's sort of like being able to borrow against the normal variations 
in wakefulness and sleep. And this is why we encourage people, if they're not training first thing in the morning, 
to push their caffeine intake out about 90 to 120 minutes after waking so they can clear some of that adenosine in 
the morning, which tends to happen even after we wake up. People can listen to episodes on Master Your Sleep or 
Perfect Sleep or the caffeine episode to understand more about that. But the final thing I just want to say here, 
and then it prompts a question, is in thinking about supplement protocols, I think a lot of people assume that once
they start taking something, they're going to have to take it all the time.

What kind of information does InsideTracker analyze from your blood and DNA to help you understand your body and 
reach your health goals?

...

In addition, please subscribe to the podcast on Apple and Spotify. And on both Apple and Spotify, you can leave us 
up to a five-star review. Please leave us any comments or suggestions or ideas for future guests or questions about
particulars of any podcast episode in the comment section on YouTube. We do read all the comments. Please also 
check out the sponsors mentioned at the beginning of today's episode and check out Athletic Greens. That's the best
way to support this podcast. In addition, please check out our Neural Network Newsletter. That is a zero cost 
newsletter available to anyone. You just provide an email and we do not share your email with anybody. You simply 
go to hubermanlab.com, you go to the menu, you look for the Neural Network Newsletter, and by signing up for the 
newsletter, you can get summaries of podcast episodes, detailed lists of protocols related to podcast episodes, 
again, all zero cost. We don't share your information. And if you'd like to see examples of previous newsletters, 
they're also available on our website, hubermanlab.com. If you're not already following us on social media, we are 
hubermanlab on both Twitter and Instagram.

How can you support the Huberman Lab podcast without incurring any cost?

Thank you for joining me for today's discussion with Dr. Reena Malik, all about urology, pelvic floor, and sexual 
health. If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's a 
terrific zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And on 
both Spotify and Apple, you can leave us up to a five-star review. If you have questions for me or comments about 
the podcast or guests that you'd like me to consider hosting on the Huberman Lab podcast, please put those in the 
comment section on YouTube. I do read all the comments. Please also check out the sponsors mentioned at the 
beginning and throughout today's episode. Thank you. many people derive tremendous benefit from them for things 
like improving sleep, hormone support, and focus. The Huberman Lab Podcast has partnered with Momentous 
Supplements. If you'd like to access the supplements discussed on the Huberman Lab Podcast, you can go to 
livemomentous, spelled O-U-S, so it's livemomentous.com slash Huberman, and you can also receive 20% off.

What are the ways to access the supplements discussed on the Huberman Lab Podcast and receive a discount?

Thank you for joining me for today's discussion with Dr. Reena Malik, all about urology, pelvic floor, and sexual 
health. If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's a 
terrific zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And on 
both Spotify and Apple, you can leave us up to a five-star review. If you have questions for me or comments about 
the podcast or guests that you'd like me to consider hosting on the Huberman Lab podcast, please put those in the 
comment section on YouTube. I do read all the comments. Please also check out the sponsors mentioned at the 
beginning and throughout today's episode. Thank you. many people derive tremendous benefit from them for things 
like improving sleep, hormone support, and focus. The Huberman Lab Podcast has partnered with Momentous 
Supplements. If you'd like to access the supplements discussed on the Huberman Lab Podcast, you can go to 
livemomentous, spelled O-U-S, so it's livemomentous.com slash Huberman, and you can also receive 20% off.

In [11]:
model_path = "Alibaba-NLP/gte-large-en-v1.5"

retriever = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path, trust_remote_code=True)
collection_name = "Huberman_gte_large_1024"
eval_results = retrieval_evaluation(dataset=golden_dataset, collection_name=collection_name, retriever=retriever, include_miss_info=True)

/Users/neerajprad/miniconda3/envs/vs-uplimit/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
/var/folders/b9/741d2y1x6pd9q8wsdzl7bbnm0000gn/T/ipykernel_40718/1823089370.py:3: ResourceWarning: unclosed <ssl.SSLSocket fd=77, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.253', 49388), raddr=('34.149.137.116', 443)>
  retriever = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path, trust_remote_code=True)
Queries: 100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


Total Processing Time: 1.27 minutes

In [12]:
print(eval_results)

(
    {
        'n': 5,
        'Retriever': 'Alibaba-NLP/gte-large-en-v1.5',
        'chunk_size': 256,
        'query_props': ['content'],
        'kw_hit_rate': 0.9,
        'kw_mrr': 0.82,
        'vector_hit_rate': 0.66,
        'vector_mrr': 0.56,
        'total_misses': 8,
        'total_questions': 100
    },
    [
        {
            'query': 'How can listeners support the podcast in addition to leaving reviews and comments on Spotify 
and Apple?',
            'kw_response': [
                {
                    'content': "Nonetheless, I think many people who are listeners of this podcast are interested 
in the things that they can do in order to inoculate themselves or at least reduce the likelihood of experiencing 
headache, especially for people who are experiencing chronic recurring headaches, such as migraine or the other 
forms of headache, which can be so debilitating. Thank you for joining me for today's discussion. If you're 
learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's the best zero cost way 
to support us. In addition, please subscribe to the podcast on Spotify and Apple. And on both Spotify and Apple, 
you can leave us up to a five-star review. If you have questions for me or comments about the podcast or guests 
that you'd like me to include on the Huberman Lab podcast, please put all of that in the comment section on 
YouTube. I do read all the comments. In addition, please check out the sponsors mentioned at the beginning and 
throughout today's episode. That's the best way to support this podcast. If you're not already following us on 
social media, we are Huberman Lab on Instagram, Twitter, Facebook, and LinkedIn.",
                    'doc_id': 'CGjdgy0cwGk_144',
                    'score': 17.35101890563965
                },
                {
                    'content': "And be sure to check out the Waking Up app that Dr. Sam Harris has made free to any
Huberman Lab listeners for 30 days by going to wakingup.com slash Huberman. Please also check out his incredible 
podcast, the Making Sense podcast. And you can find any number of Sam Harris's different books on meditation, 
consciousness, philosophy, neuroscience, politics, and more. You can find links to those books by going to 
samharris.org. If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. 
That's the best zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And
on both Spotify and Apple, you can leave us up to a five-star review. If you have questions for us or comments or 
topics that you'd like me to cover, or guests you'd like me to invite onto the Huberman Lab podcast, please put 
those in the comment section on YouTube. I do read all the comments. Please also check out the sponsors mentioned 
at the beginning and throughout today's episode. That's the best way to support this podcast.",
                    'doc_id': '-wIt_WsJGfw_228',
                    'score': 16.914188385009766
                },
                {
                    'content': "Susanna Soberg. If you'd like to learn more about Dr. Soberg's research, or you 
would like to learn about the research of her institute, the Soberg Institute, please see the links in the show 
note caption. Also in the show note caption, you can find a link to Dr. Soberg's excellent book, Winter Swimming. 
If you're learning from and or enjoying this podcast, please subscribe to our YouTube channel. That's a terrific 
zero cost way to support us. In addition, please subscribe to the podcast on Spotify and Apple. And in addition, on
both Spotify and Apple, you can leave us up to a five-star review. If you have questions for me or topics you'd 
like me to cover on the Huberman Lab podcast, or guests that you'd like me to consider inviting on the Huberman Lab
podcast, please put that in the comments on YouTube. I do read all the comments. In addition, pl

# Conclusion
***

We now have a way of measuring the performance of our retrieval system.  This will allow you to make tweaks/changes to the system and then be able to objectively tell whether or not the tweak/change improved or degraded its performance.  Here are a few things to consider going forward:  

- Keep in mind what the ulitmate goal of the system is that you are building.  For this course, we are trying to retrieve the most relevant documents as possible that will effectively address a user query, assuming the information is found within the corpus.  This means that we don't need pages and pages of relevant results, we actually only need the top 3-5, just enough to allow our Reader (the OpenAI LLM) to answer the user query.  This is an important point to be thinking about as you are making changes to the retrieval system.
- Feel free to set the `include_miss_info` param to `True`.  Doing so will return a list of both the keyword and vector responses that did not contain the relevant `doc_id` (a "total_miss" means the `doc_id` was not present in either the `kw_doc_ids` or the `vector_doc_ids`).  Take a look at the style of the queries being asked and compare them with the returned responses.  Why are those responses being returned?  Are they close to the intent of the query?
- Last but not least, you are now free to make changes to your system to improve the `hit_rate` and `mrr` scores.  If it were me, I'd start with switching out to a more performant [embedding model](https://huggingface.co/spaces/mteb/leaderboard).  There will be more opportunities to pick up some low hanging fruit, but we'll have to wait until the following week when hybrid search and Rerankers are introduced.  Whatever you do though, don't change params for the `SentenceSplitter` that you use for chunking the corpus.  Due to the way the golden dataset is derived, it's unfortunately dependent on those original `SentenceSplitter` settings remaining the same across evaluations. That is, of course, unless you want to build out your own golden dataset....

# *** OPTIONAL: Chunk Size Evaluation ***

In our initial Notebook we created a dataset with a chunk size of 256.  In order to evaluat the impact that chunk size has on retrieval for both search methods, it's a useful exercise to execute the `retrieval_evaluation` function on datasets of multiple chunk sizes.  In order to accomplish that follow these simple steps:
- Bust out the `create_dataset` function that you created in Assignment 1.4.  Create datasets of chunks lengths **128** and **512**.  **Ensure that you set the `chunk_overlap` param to zero for each run.  Golden datasets of corresponding chunks lengths have already been created for you in the `data/golden_datasets` directory.
- Index those datasets on Weaviate ensuring that you stick to the standard naming convention as discussed in Notebook 2 i.e `"Huberman_minilm_<chunk_size>"`
- Evaluate results using the chunk sizes as a parameter, see example code below:

### Sample code for automated chunk_size evaluation

In [16]:
all_results = []
for size in [128, 256, 512]:

    #load golden datasets
    data_path = f'../data/golden_datasets/golden_{size}.json'
    golden_dataset = FileIO().load_json(data_path)

    #assign collection name
    collection_name = f'Huberman_minilm_{size}'
    print(f'Running test on chunk size {size} on {collection_name} Collection')

    #get results by chunk_size
    results = retrieval_evaluation(golden_dataset, collection_name, client, query_properties=['content'], chunk_size=size)
    all_results.append(results)

Running test on chunk size 128 on Huberman_minilm_128 Collection

Queries: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.08it/s]


Total Processing Time: 0.33 minutes

Running test on chunk size 256 on Huberman_minilm_256 Collection

Queries: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.12it/s]


Total Processing Time: 0.33 minutes

Running test on chunk size 512 on Huberman_minilm_512 Collection

Queries: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.10it/s]


Total Processing Time: 0.33 minutes

In [21]:
print(all_results)